# Lecture 09: Softmax Classifier

## MNIST Network

https://github.com/pytorch/examples/blob/master/mnist/main.py

In [ ]:
import numpy as np
import torch
from torch import cuda
from torch.autograd import Variable
from torch.utils.data import Dataset, DataLoader
import torch.nn.functional as F
from torchvision import datasets
import torchvision.transforms as transforms

In [ ]:
# MNIST Dataset
train_dataset = datasets.MNIST(root="./data/",
                               train=True,
                               transform=transforms.ToTensor(),
                               download=True)

test_dataset = datasets.MNIST(root="./data/",
                              train=False,
                              transform=transforms.ToTensor())

In [ ]:
batch_size = 32
device = 'cuda' if cuda.is_available() else 'cpu'
print(f'Training MNIST Model on {device}\n{"=" * 44}')

Training MNIST Model on cpu


In [ ]:
# Data Loader (Input Pipeline)
train_loader = DataLoader(dataset=train_dataset,
                          batch_size=batch_size,
                          shuffle=True)
test_loader = DataLoader(dataset=test_dataset,
                         batch_size=batch_size,
                         shuffle=False)

In [ ]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.l1 = nn.Linear(784, 520)
        self.l2 = nn.Linear(520, 320)
        self.l3 = nn.Linear(320, 240)
        self.l4 = nn.Linear(240, 120)
        self.l5 = nn.Linear(120, 10)

    def forward(self, x):
        x = x.view(-1, 784) # Flatten the data (n, 1, 28, 28) => (n, 784)
        x = F.relu(self.l1(x))
        x = F.relu(self.l2(x))
        x = F.relu(self.l3(x))
        x = F.relu(self.l4(x))
        return self.l5(x) # No need activation

In [ ]:
model = Net()
model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.01, momentum=0.5)

In [ ]:
def train(epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()

        if batch_idx%100 == 0:
            print(f"Train Epoch: {epoch} [{batch_idx * len(data)}/{len(train_loader.dataset)} ({(100. * batch_idx / len(train_loader)):.6f})%]\tLoss: {loss.data.item():.6f}")

In [ ]:
def test():
    model.eval()
    test_loss = 0
    correct = 0

    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)

            # sum up batch loss
            test_loss += criterion(output, target).item()

            # get the index of the max log-probability
            pred = torch.max(output.data, 1)[1]
            # pred = output.data.max(1, keepdim=True)[1]
            correct += pred.eq(target.data.view_as(pred)).cpu().sum()

    test_loss /= len(test_loader.dataset)
    print(f"Test set : Average loss = {test_loss:.4f}, Accuracy = {correct}/{len(test_loader.dataset)} ({(100. * correct / len(test_loader.dataset)):.0f}%)\n\n")

In [ ]:
epochs = 10
for epoch in range(1, epochs+1):
    train(epoch)
    test()

Train Epoch: 1 [0/60000 (0.000000)%]	Loss: 2.308950
Train Epoch: 1 [3200/60000 (5.333333)%]	Loss: 2.295634
Train Epoch: 1 [6400/60000 (10.666667)%]	Loss: 2.292025
Train Epoch: 1 [9600/60000 (16.000000)%]	Loss: 2.307748
Train Epoch: 1 [12800/60000 (21.333333)%]	Loss: 2.288687
Train Epoch: 1 [16000/60000 (26.666667)%]	Loss: 2.269932
Train Epoch: 1 [19200/60000 (32.000000)%]	Loss: 2.268586
Train Epoch: 1 [22400/60000 (37.333333)%]	Loss: 2.222619
Train Epoch: 1 [25600/60000 (42.666667)%]	Loss: 2.176989
Train Epoch: 1 [28800/60000 (48.000000)%]	Loss: 1.851439
Train Epoch: 1 [32000/60000 (53.333333)%]	Loss: 1.539903
Train Epoch: 1 [35200/60000 (58.666667)%]	Loss: 0.949393
Train Epoch: 1 [38400/60000 (64.000000)%]	Loss: 0.679966
Train Epoch: 1 [41600/60000 (69.333333)%]	Loss: 0.877325
Train Epoch: 1 [44800/60000 (74.666667)%]	Loss: 0.566020
Train Epoch: 1 [48000/60000 (80.000000)%]	Loss: 0.568280
Train Epoch: 1 [51200/60000 (85.333333)%]	Loss: 0.452411
Train Epoch: 1 [54400/60000 (90.666667)%